In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
    dff = pd.read_csv('https://raw.githubusercontent.com/data301-2021-summer2/project-group41-project/main/data/processed/FiresNEW.csv?token=AUXSV2UTBCDHPSFSJ7M4UZTBA4RDU', encoding = 'ISO-8859-1', low_memory=False)
    dff = (
        dff.drop(columns=["FOD_ID","FPA_ID","SOURCE_SYSTEM_TYPE","SOURCE_SYSTEM","NWCG_REPORTING_AGENCY","NWCG_REPORTING_UNIT_ID","NWCG_REPORTING_UNIT_NAME","SOURCE_REPORTING_UNIT","SOURCE_REPORTING_UNIT_NAME","LOCAL_FIRE_REPORT_ID","LOCAL_INCIDENT_ID","FIRE_CODE","FIRE_NAME","ICS_209_PLUS_INCIDENT_JOIN_ID","ICS_209_PLUS_COMPLEX_JOIN_ID","MTBS_ID","MTBS_FIRE_NAME","COMPLEX_NAME","DISCOVERY_DATE","DISCOVERY_DOY","DISCOVERY_TIME","NWCG_GENERAL_CAUSE","NWCG_CAUSE_AGE_CATEGORY","CONT_DATE","CONT_DOY","CONT_TIME","LATITUDE","LONGITUDE","OWNER_DESCR","COUNTY","FIPS_CODE","FIPS_NAME"])
        .drop(dff[dff.NWCG_CAUSE_CLASSIFICATION == 'Human'].index)
        .drop(dff[dff.NWCG_CAUSE_CLASSIFICATION == 'Missing data/not specified/undetermined'].index)
        .dropna()
        .drop(dff[(dff.FIRE_SIZE_CLASS == 'A')].index)
        .drop(columns=['NWCG_CAUSE_CLASSIFICATION'])
    )
dff.head()


KeyError: "['FOD_ID' 'FPA_ID' 'SOURCE_SYSTEM_TYPE' 'SOURCE_SYSTEM'\n 'NWCG_REPORTING_AGENCY' 'NWCG_REPORTING_UNIT_ID'\n 'NWCG_REPORTING_UNIT_NAME' 'SOURCE_REPORTING_UNIT'\n 'SOURCE_REPORTING_UNIT_NAME' 'LOCAL_FIRE_REPORT_ID' 'LOCAL_INCIDENT_ID'\n 'FIRE_CODE' 'FIRE_NAME' 'ICS_209_PLUS_INCIDENT_JOIN_ID'\n 'ICS_209_PLUS_COMPLEX_JOIN_ID' 'MTBS_ID' 'MTBS_FIRE_NAME' 'COMPLEX_NAME'\n 'DISCOVERY_DATE' 'DISCOVERY_DOY' 'DISCOVERY_TIME' 'NWCG_GENERAL_CAUSE'\n 'NWCG_CAUSE_AGE_CATEGORY' 'CONT_DATE' 'CONT_DOY' 'CONT_TIME' 'LATITUDE'\n 'LONGITUDE' 'OWNER_DESCR' 'COUNTY' 'FIPS_CODE' 'FIPS_NAME'] not found in axis"

In [19]:
    dfstate = pd.read_csv('https://raw.githubusercontent.com/data301-2021-summer2/project-group41-project/main/data/processed/csvData.csv?token=AI4GJYBHQ5NOQRGXEA722W3BA4SNK')
    dfstate = (
        dfstate.drop(columns='Abbrev')
        .rename(columns={'Code':'STATE'})
    )
    
    
   

,State,STATE
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [16]:
    df3 = (
        dff.set_index('STATE').combine_first(dfstate.set_index('STATE'))
        .reset_index(inplace=True)
        .drop(columns='STATE')
        .dropna()
        .rename(columns={'FIRE_YEAR':'Date','State':'Location'})
    )
    df2['Date'] = df2['Date'].astype(int)


KeyError: "None of ['STATE'] are in the columns"

In [26]:
dftemp=pd.read_csv('https://raw.githubusercontent.com/data301-2021-summer2/project-group41-project/main/data/processed/tavg.csv?token=AI4GJYEE7OU2ZA6CF6XEIE3BA4S3W')
f = lambda dftemp : dftemp['Location'].split("CD")
dftemp['Location'] = dftemp.apply(f, axis=1)
g = lambda dftemp : dftemp['Location'].pop(0)
dftemp['Location'] = dftemp.apply(g, axis=1)
dftemp.reset_index()
dftemp = dftemp.drop(columns=['Location ID','Rank','Anomaly (1901-2000 base period)','1901-2000 Mean'])
dftemp['Location'] = dftemp['Location'].str.rstrip()
dftemp['Celsius'] = (dftemp['Value']-32)*5/9
dftemp['Celsius'] = dftemp['Celsius'].round(1)
dftemp = dftemp.drop(columns=['Value'])
dftemp = dftemp.drop(dftemp[dftemp['Date'] % 10 != 7].index)
dftemp['Date'] = dftemp['Date']//100
dftemp['Date'] = dftemp['Date'].astype(int)
dftemp = dftemp.drop(dftemp[dftemp.Date > 2018].index)
dftemp = dftemp.drop(dftemp[dftemp['Date'] < 1992].index)
dftemp = dftemp.reset_index(drop = True)
dftemp = dftemp.reset_index(drop = False)



dffinal = (
    df2.groupby(['Date','Location']).size()
     .to_frame()
    .reset_index(inplace = True)
    .drop(dffinal[dffinal.Location == 'Alaska'].index)
    .drop(dffinal[dffinal.Location == 'Hawaii'].index)
    .sort_values(by=['Location','Date'])
    .rename(columns={0: '# of Fires'})
    .reset_index(drop=True)
     .reset_index(drop=False)
    )

d1=dffinal
d1 = d1.drop(columns=['index'])
d1['Date']=d1['Date'].values.astype(str)
d1['Location']=d1['Location'].values.astype(str)
d1['id'] = d1['Date'].str.cat(d1['Location'],sep='-')
d1


dftemp12 = dftemp
dftemp12 = dftemp12.drop(columns=['index'])
dftemp12['Date']=dftemp12['Date'].values.astype(str)
dftemp12['Location']=dftemp12['Location'].values.astype(str)
dftemp12['id'] = dftemp12['Date'].str.cat(dftemp12['Location'],sep='-')
dftemp100 = dftemp12.drop(columns=['Date','Location'])
dftemp100

dffinal = pd.merge(d1,dftemp100,how='right',on='id')
dffinal = dffinal.drop(columns=['id'])

dffinal.head()

NameError: name 'df2' is not defined